In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt

vamos carregar os dados

In [3]:
df_limites = pd.read_excel('gastos_limites.xlsx', sheet_name='limites')
df_gastos = pd.read_excel('gastos_limites.xlsx', sheet_name='gastos')

Limpar os dados do limite

In [4]:
df_limites['dt_fechamento'] = pd.to_datetime(df_limites.dt_fechamento)
df_limites['limite'] = df_limites['limite'].str.replace('$','')
df_limites['limite'] = df_limites['limite'].str.replace('R ','')
df_limites['limite'] = df_limites['limite'].str.replace(',','.')
df_limites['limite'] = pd.to_numeric(df_limites['limite'])
df_limites['cliente'] = df_limites['cliente'].str.title()

<ipython-input-4-378fc3bdbaeb>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_limites['limite'] = df_limites['limite'].str.replace('$','')


In [5]:
df_limites.isnull().sum()

Unnamed: 0       0
cliente          0
limite           0
dt_fechamento    0
dtype: int64

Limpar os dados dos gastos

In [6]:
df_gastos['dt_operacao'] = pd.to_datetime(df_gastos.dt_operacao)
df_gastos['valor_operacao'] = df_gastos['valor_operacao'].str.replace('$','')
df_gastos['valor_operacao'] = df_gastos['valor_operacao'].str.replace('R ','')
df_gastos['valor_operacao'] = df_gastos['valor_operacao'].str.replace(',','.')
df_gastos['valor_operacao'] = pd.to_numeric(df_gastos['valor_operacao']) 
df_gastos['mes_atual'] = pd.to_datetime(df_gastos.mes_atual, format='%b/%y')
df_gastos['nm_cliente'] = df_gastos['nm_cliente'].str.title()

<ipython-input-6-ce37e9d1b2cf>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_gastos['valor_operacao'] = df_gastos['valor_operacao'].str.replace('$','')


In [7]:
inverteNome = lambda s : s.split(', ')[1] + ' ' +  s.split(', ')[0] if ', ' in s else s


In [8]:
df_gastos['nm_cliente'] = df_gastos['nm_cliente'].apply(inverteNome)
df_limites['cliente'] = df_limites['cliente'].apply(inverteNome)

In [9]:
df_gastos.isnull().sum()

Unnamed: 0        0
nm_cliente        0
valor_operacao    0
num_parcelas      0
dt_operacao       0
mes_atual         0
dtype: int64

In [10]:
df_limites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     20 non-null     int64         
 1   cliente        20 non-null     object        
 2   limite         20 non-null     float64       
 3   dt_fechamento  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 624.0+ bytes


In [11]:
df_limites.head()

,Unnamed: 0,cliente,limite,dt_fechamento
0,0,João Silva,2575.0,2021-01-25
1,1,José Silva,4500.0,2021-05-01
2,2,Antônio Oliveira,2325.0,2021-01-25
3,3,Francisco Santos,375.0,2021-01-25
4,4,Carlos Rodrigues,2675.0,2021-05-01


In [12]:
df_gastos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0      150 non-null    int64         
 1   nm_cliente      150 non-null    object        
 2   valor_operacao  150 non-null    float64       
 3   num_parcelas    150 non-null    int64         
 4   dt_operacao     150 non-null    datetime64[ns]
 5   mes_atual       150 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 6.5+ KB


In [15]:
df_gastos['dt_operacao'].unique()

array(['2021-01-01T00:00:00.000000000', '2021-02-01T00:00:00.000000000',
       '2021-03-01T00:00:00.000000000', '2021-04-01T00:00:00.000000000',
       '2021-05-01T00:00:00.000000000', '2021-06-01T00:00:00.000000000',
       '2021-07-01T00:00:00.000000000', '2021-08-01T00:00:00.000000000',
       '2021-09-01T00:00:00.000000000', '2021-10-01T00:00:00.000000000',
       '2021-11-01T00:00:00.000000000', '2021-12-01T00:00:00.000000000',
       '2021-01-13T00:00:00.000000000', '2021-01-14T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000', '2021-01-16T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-19T00:00:00.000000000', '2021-01-20T00:00:00.000000000',
       '2021-01-21T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000',
       '2021-01-25T00:00:00.000000000', '2021-01-26T00:00:00.000000000',
       '2021-01-27T00:00:00.000000000', '2021-01-28

In [29]:
df_gastos_acum = df_gastos.groupby(df_gastos['dt_operacao'].dt.strftime('%B'))[['nm_cliente','valor_operacao']]

In [30]:
df_gastos_acum.head()

,nm_cliente,valor_operacao
0,Paulo Gomes,855.00
1,Camila Santos,504.18
2,Márcia Silva,970.00
3,Maria Alves,2010.31
4,Lucas Oliveira,465.00
5,Paulo Gomes,465.50
6,Sandra Souza,2107.00
7,Maria Alves,1071.67
8,Maria Alves,382.67
9,Francisco Santos,889.00


In [20]:
df_gastos_acum.pivot_table(values='valor_operacao', index='nm_cliente', columns='dt_operacao')

KeyError: 'nm_cliente'